# Costweet : CNN + vectorisation manuelle

Ce notebook classe des phases en 3 catégories "cher", "moyen","éco".

Il fait partie d'une série de notebook permettant de comparer différentes approches de classification en faisant varier les architectures de réseaux de neurones et les méthodes d'encodification des textes.

Il s'appuie sur un ensemble de données de petite taille.

Cet ensemble est vectorisé selon une approche spécifique qui comprime les phrases à l'aide d'un vocabulaire réduit choisit manuellement et non via l'usage d'un algorithme supervisé.

Le réseau de nerones est architecturé de trois couches de type convolution.

L'accuracy obtenu le 7 février 2018  est de 92,7% à comparer aux 85% obtenus avec avec un réseau sans convolution.
Accuracy de 94% obtenue le 9 février 2018 en quarante epochs.

La vitesse d'apprentissage est de 1ms par step et on a besoin d'une quarantainee d'epochs pour dépasser 90%.

Keras uses tensorflow uses sklearn uses numpy and pandas

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Convolution1D, Flatten, Dropout, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder



Using TensorFlow backend.


 DATA PROCESS
 les données ont été saisies sous excel et sauvegardées en csv utf-8 bizarement avec l'option séparateur virgules de microsoft
 


In [2]:
def getDataset():
	dataset = pd.read_csv("RuleCout2.csv", delimiter=";", encoding='utf-8')
	#print ( 'dataset loaded with shape',dataset.shape)
	X = dataset['text']
	Y = dataset['class']
	dataset.set_index('text') #pour avoir les classes dans un ordre quelconque
	return dataset, X, Y

# utile pour les colonnes sur une ligne et afficher toutes les lignes
pd.set_option('display.width', 200)
pd.set_option('display.max_rows', 1000)

# on ajoutera peu à peu de nouvelles colonnes au dataset 
dataset, X, Y = getDataset()
print(dataset)

                         text  class
0             à prix modique     éco
1     aucun problème d'argent   cher
2                 avantageux     éco
3               bas de gamme     éco
4           bistrot pas cher     éco
5                         bon   cher
6                 bon marché     éco
7   bon rapport qualité prix   moyen
8                       cher    cher
9                         éco    éco
10                économique     éco
11             entre 10 et 20    éco
12             gastronomique    cher
13             haut de gamme    cher
14           j’ai les moyens    cher
15             je m’en fiche    cher
16              je m’en fous    cher
17             je suis riche    cher
18          jusqu'à cinquante   cher
19  le meilleur des meilleurs   cher
20                      luxe    cher
21                   luxueux    cher
22      ma bourse est pleine    cher
23     menu pas cher le midi     éco
24                   modique     éco
25       moins de cent euros    cher
2

#### Les mots les plus signifiants ont été classés par catégorie.

La catégorie "éco" indique les mots qui vont dans le sens de choix économiques
La catégorie "troquet" indique des classes de restaurant
La catégorie "négation" indique des mots qui inverse le sens
La catégorie "éco" indique les mots qui vont dans le sens de choix économiques
La catégorie "monnaie" indique des quantités monayables
La catégorie "ambigu" contient des termes qui se caractérisent par leur absence de positionnenement qui est une information en soit
Les catégories "nombres" sont explicites.

Les mots les plus signifiants vont permettre un encdage par catégorie.
ansi "pas trop cher" deviendra "négation 0 cher" ou [5 0 10]. 
On se posera la question de la pérsence du zéro : est-elle bénéfique ou néfaste ?

#### Quel est l'impact de ce travail de répartition sur  le taux de réussite ?



In [3]:
eco = ['modique','économique','ténu','japonais', 'chinois','kebab', 'self','pourri','avantageux','marché','bas',
       'plein','petit','petite']
troquet = ['troquet','bistrot','cantine','self','brasserie','restaurant','restau','resto']
negation = ['pas','ni','peu']
cher = ['cher','bon','haut','luxe','luxueux','fiche','fous','moyens', 'gastronomique', 'classe','riche','michelin','etoile','étoiles',
'standing','haut','chicos','onéreux','onereux','super''meilleur','meilleurs','grattin','gratin', 'bien']
monnaie = ['gamme', 'euro', 'prix', 'euros','problème','pb','qualité']
ambigu = ['milieu','moyen']
nombreEco = ['0','5','10','15','zéro','cinq','dix','quinze']
nombreCher= ['20','30','40','50','80','100','cent','vingt','trente','quarante','cinquante','quatre-vingt','cent']
limitatif = ['moins', 'peu','jusqu''à','plutôt']

# le choix de l'entier n'importe pas sauf pour le zéro qui est la valeur complémentée par pad_sequences
def score(word):
	if word in eco: 
		return 1
	elif word in nombreEco:
		return 2
	elif word in troquet: 
		return 3
	elif word in negation:
		return 4
	elif word in limitatif:
		return 5
	elif word in monnaie:
		return 6
	elif word in ambigu:
		return 7
	elif word in nombreCher:
		return 8
	elif word in cher:
		return 9
	else: 
		return 0
	
print ( 'score modique', score('modique'))
print ( 'score troquet', score('troquet'))
print ( 'score inconnu', score('inconnu'))


score modique 1
score troquet 3
score inconnu 0


In [4]:
#ai préféré utiliser le tokenizer de keras plutôt que celui de nltk car les valeurs filtrées par défaut comprennent l'apostrophe
def tokenize(sentence):
	tokens = text_to_word_sequence(sentence)
	return tokens
	
dataset['tokens']= dataset['text'].map(tokenize)
dataset['tokens']

0                 [à, prix, modique]
1        [aucun, problème, d'argent]
2                       [avantageux]
3                   [bas, de, gamme]
4               [bistrot, pas, cher]
5                              [bon]
6                      [bon, marché]
7      [bon, rapport, qualité, prix]
8                             [cher]
9                              [éco]
10                      [économique]
11               [entre, 10, et, 20]
12                   [gastronomique]
13                 [haut, de, gamme]
14               [j’ai, les, moyens]
15                 [je, m’en, fiche]
16                  [je, m’en, fous]
17                 [je, suis, riche]
18              [jusqu'à, cinquante]
19    [le, meilleur, des, meilleurs]
20                            [luxe]
21                         [luxueux]
22         [ma, bourse, est, pleine]
23       [menu, pas, cher, le, midi]
24                         [modique]
25          [moins, de, cent, euros]
26           [moins, de, dix, euros]
2

L'étape suivante consiste à transformer une liste de tokens en une liste de nombres à encoder.
Comme la taille de cette liste doit être fixe on utilise pad_sequence qui ajoute des zéros ou enlève des nombres. 

In [5]:
def token2code(tokens):
	return list(map(score,tokens))

#dataset['Xcodes'] = list (map(token2code,dataset['tokens']))
def encode(data):
	return dataset['tokens'].map(token2code)

Xcodes = encode(dataset)
XEncoded = pad_sequences(Xcodes,maxlen=5,padding='pre',dtype='float')
dataset['XEncoded'] = XEncoded.tolist()
dataset


,text,class,tokens,XEncoded
0,à prix modique,éco,"[à, prix, modique]","[0.0, 0.0, 0.0, 6.0, 1.0]"
1,aucun problème d'argent,cher,"[aucun, problème, d'argent]","[0.0, 0.0, 0.0, 6.0, 0.0]"
2,avantageux,éco,[avantageux],"[0.0, 0.0, 0.0, 0.0, 1.0]"
3,bas de gamme,éco,"[bas, de, gamme]","[0.0, 0.0, 1.0, 0.0, 6.0]"
4,bistrot pas cher,éco,"[bistrot, pas, cher]","[0.0, 0.0, 3.0, 4.0, 9.0]"
5,bon,cher,[bon],"[0.0, 0.0, 0.0, 0.0, 9.0]"
6,bon marché,éco,"[bon, marché]","[0.0, 0.0, 0.0, 9.0, 1.0]"
7,bon rapport qualité prix,moyen,"[bon, rapport, qualité, prix]","[0.0, 9.0, 0.0, 6.0, 6.0]"
8,cher,cher,[cher],"[0.0, 0.0, 0.0, 0.0, 9.0]"
9,éco,éco,[éco],"[0.0, 0.0, 0.0, 0.0, 0.0]"


Le prochain pavé permet de transformer les catégories "éco","moyen" et cher en vecteur.
Ces vecteurs sont rangés dans la colonne 'Ycodes' du dataset

In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dataset['Ycodes'] = pd.Series(list(dummy_y))
dataset


,text,class,tokens,XEncoded,Ycodes
0,à prix modique,éco,"[à, prix, modique]","[0.0, 0.0, 0.0, 6.0, 1.0]","[0.0, 0.0, 1.0]"
1,aucun problème d'argent,cher,"[aucun, problème, d'argent]","[0.0, 0.0, 0.0, 6.0, 0.0]","[1.0, 0.0, 0.0]"
2,avantageux,éco,[avantageux],"[0.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]"
3,bas de gamme,éco,"[bas, de, gamme]","[0.0, 0.0, 1.0, 0.0, 6.0]","[0.0, 0.0, 1.0]"
4,bistrot pas cher,éco,"[bistrot, pas, cher]","[0.0, 0.0, 3.0, 4.0, 9.0]","[0.0, 0.0, 1.0]"
5,bon,cher,[bon],"[0.0, 0.0, 0.0, 0.0, 9.0]","[1.0, 0.0, 0.0]"
6,bon marché,éco,"[bon, marché]","[0.0, 0.0, 0.0, 9.0, 1.0]","[0.0, 0.0, 1.0]"
7,bon rapport qualité prix,moyen,"[bon, rapport, qualité, prix]","[0.0, 9.0, 0.0, 6.0, 6.0]","[0.0, 1.0, 0.0]"
8,cher,cher,[cher],"[0.0, 0.0, 0.0, 0.0, 9.0]","[1.0, 0.0, 0.0]"
9,éco,éco,[éco],"[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]"


###MODEL DEFINITION
Le réseau comprend trois couches de type convolution. Une seule suffirait ?
La couche embedding contient un noeud par élément du vocabulaire, le vocabulaire comprend 10 mots (un par classe).

La dernière est de type softmax et comprend trois noeuds.

Chaque phrase est représentée à l'aide d'un vecteur de longueur cinq à l'aide d'un vocabulaire à neuf mots.

In [7]:
########################### MODEL DEFINITION
input_dim = 10 #size of vocabular
output_dim = 1 #embedding_vecor_length
input_length = 5 #matrice width

def sequential_model():
    model = Sequential()
    model.add(Embedding(input_dim, output_dim, input_length=input_length))
    #model.add(Convolution1D(64, 3, border_mode='same'))
    model.add(Convolution1D(64, 3, padding='same'))
    model.add(Convolution1D(32, 3, padding='same'))
    model.add(Convolution1D(16, 3, padding='same'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(180,activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



model = sequential_model()
model

In [8]:
###### TRAINING + EVALUATION
x_train = XEncoded
y_train = dummy_y
batch_size = 5
model.fit(x_train, y_train,
          epochs=40,
          batch_size=batch_size)
x_test = x_train
y_test = y_train
loss,metric = model.evaluate(x_test, y_test, batch_size=batch_size)
print ("Loss = ", loss," Accuracy = ",metric)

Epoch 1/40
73/73 [==============================] - 1s 9ms/step - loss: 0.6187 - acc: 0.6667
Epoch 2/40
73/73 [==============================] - 0s 644us/step - loss: 0.5579 - acc: 0.6621
Epoch 3/40
73/73 [==============================] - 0s 753us/step - loss: 0.5661 - acc: 0.6575
Epoch 4/40
73/73 [==============================] - 0s 808us/step - loss: 0.5880 - acc: 0.6484
Epoch 5/40
73/73 [==============================] - 0s 740us/step - loss: 0.5920 - acc: 0.6393
Epoch 6/40
73/73 [==============================] - 0s 795us/step - loss: 0.5907 - acc: 0.6119
Epoch 7/40
73/73 [==============================] - 0s 740us/step - loss: 0.5434 - acc: 0.7260
Epoch 8/40
73/73 [==============================] - 0s 753us/step - loss: 0.5357 - acc: 0.6986
Epoch 9/40
73/73 [==============================] - 0s 767us/step - loss: 0.5081 - acc: 0.7626
Epoch 10/40
73/73 [==============================] - 0s 753us/step - loss: 0.4561 - acc: 0.7900
Epoch 11/40
73/73 [==============================] 

In [9]:
dataset['predicted'] = list(model.predict(XEncoded)) 

def decodeY(L):
	if L[0]>L[1] and L[0]>L[2]: 
		return "cher"
	elif L[2]>L[1] and L[2]>L[0]:
		return "éco"
	else:
		return "moyen"
	
dataset['resultat']=list(map(decodeY,dataset['predicted']))
dataset


,text,class,tokens,XEncoded,Ycodes,predicted,resultat
0,à prix modique,éco,"[à, prix, modique]","[0.0, 0.0, 0.0, 6.0, 1.0]","[0.0, 0.0, 1.0]","[0.0726503, 3.4266166e-05, 0.9273155]",éco
1,aucun problème d'argent,cher,"[aucun, problème, d'argent]","[0.0, 0.0, 0.0, 6.0, 0.0]","[1.0, 0.0, 0.0]","[0.6426059, 0.0005472064, 0.35684687]",cher
2,avantageux,éco,[avantageux],"[0.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.047711007, 2.6958018e-05, 0.95226204]",éco
3,bas de gamme,éco,"[bas, de, gamme]","[0.0, 0.0, 1.0, 0.0, 6.0]","[0.0, 0.0, 1.0]","[0.4843233, 0.00018078761, 0.5154959]",éco
4,bistrot pas cher,éco,"[bistrot, pas, cher]","[0.0, 0.0, 3.0, 4.0, 9.0]","[0.0, 0.0, 1.0]","[0.5483598, 0.000302679, 0.45133752]",cher
5,bon,cher,[bon],"[0.0, 0.0, 0.0, 0.0, 9.0]","[1.0, 0.0, 0.0]","[0.90271896, 0.00020524234, 0.09707582]",cher
6,bon marché,éco,"[bon, marché]","[0.0, 0.0, 0.0, 9.0, 1.0]","[0.0, 0.0, 1.0]","[0.061124004, 3.0187624e-05, 0.9388458]",éco
7,bon rapport qualité prix,moyen,"[bon, rapport, qualité, prix]","[0.0, 9.0, 0.0, 6.0, 6.0]","[0.0, 1.0, 0.0]","[0.026777001, 0.7363432, 0.23687986]",moyen
8,cher,cher,[cher],"[0.0, 0.0, 0.0, 0.0, 9.0]","[1.0, 0.0, 0.0]","[0.90271896, 0.00020524234, 0.09707582]",cher
9,éco,éco,[éco],"[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.6345933, 0.0001430715, 0.3652636]",cher


In [10]:
print ("Loss = ", loss," Accuracy = ",metric)

Loss =  0.2407057298576995  Accuracy =  0.9132420183861092
